In [31]:
import time
from time import sleep
from selenium import webdriver #qué navegador vamos a usar
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [32]:
#como el navegador a usar es chrome, hemos descargado ya el driver de la página, guardamos la ruta en una variable
chrome_path = 'chromedriver_linux64/chromedriver'
driver = webdriver.Chrome(executable_path = chrome_path)

In [33]:
url = 'https://www.stylevana.com/en_EU/skincare/facial-cleanser/face-wash-cleansers.html' #página a scrappear. En Jolse no están listados los ingredientes
driver.get(url)

¿Qué queremos obtener de la página?
1. Nombre del producto
2. Precio (?)
3. Detalles
4. Ingredientes

En Stylevana, los productos están en contenedores <li class=item product-item>

In [34]:
#cleansers = driver.find_elements_by_xpath('//li[@class="item product-item"]') #extrae una lista de todos los elementos cuyo atributo sea el especificado tras @

In [35]:
load_more = driver.find_element_by_xpath('//a[@class="next i-next"]')

In [36]:
for i in range(2):
    try:
        cleansers = driver.find_elements_by_xpath('//li[@class="item product-item"]')
        load_more.click()
        load_more = driver.find_element_by_xpath('//a[@class="next i-next"]')
    except:
        break

no se, no funciona
while True:
    #extrae una lista de todos los elementos cuyo atributo sea el especificado tras @
    cleansers = driver.find_elements_by_xpath('//li[@class="item product-item"]')
    name = []
    for cleanser in cleansers:
        name.append(cleanser.text)
        for i in range(3):
            try:
                load_more.click()
            #sleep(random.uniform(7.0, 10.0))
                load_more = driver.find_element_by_xpath('//a[@class="next i-next"]')
            except:
                break

In [ ]:
#for i in range(3):
    #try:
        
      #  load_more.click()
        #sleep(random.uniform(7.0, 10.0))
    #    load_more = driver.find_element_by_xpath('//a[@class="next i-next"]')
    #except:
     #   break

la info de los productos (el nombre es lo que nos interesa), se encuentra dentro de un <div class="product-item-details"

In [37]:
for cleanser in cleansers:
    # . para que busque dentro del elemento (en este caso cleansers, // todos los elementos)
    name = cleanser.find_element_by_xpath('.//h2[@class="product-name"]').text
    print(name)

iUNIK - Calendula Complete Cleansing Oil - 200ml
Krave - Matcha Hemp Hydrating Cleanser 120ml
COSRX - Low pH Good Morning Gel Cleanser - 150ml
SOME BY MI - AHA-BHA-PHA 30days Miracle Acne Clear Body Cleanser - 400g
heimish - All Clean Balm
Isntree - Micellar Washing Cleansing Water - 300ml
BEAUTY OF JOSEON - Cleansing Balm - 80g
iUNIK - Centella Bubble Cleansing Foam - 150ml
Haruharu WONDER - Black Rice Moisture Deep Cleansing Oil - 150ml
Haruharu WONDER - Black Rice Moisture 5.5 Soft Cleansing Gel - 100ml
heimish - All Clean Green Foam - 150ml
SOME BY MI - Bye Bye Blackhead 30days Miracle Green Tea Tox Bubble Cleanser - 120g


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[5]/div/div/div[1]/div/div/button"}
  (Session info: chrome=87.0.4280.66)


In [ ]:


# initiate empty dataframe
df = pd.DataFrame(columns=['Label', 'URL'])
print(df)

# step 1
tickers = ['moisturizing-cream-oils-mists', 'cleanser', 'facial-treatments', 'face-mask',
           'eye-treatment-dark-circle-treatment', 'sunscreen-sun-protection']

for ticker in tickers:
    url = 'https://www.sephora.com/shop/' + ticker + '?pageSize=300'
    driver.get(url)

    xpath = '/html/body/div[5]/div/div/div[1]/div/div/button'
    btn = driver.find_element_by_xpath(xpath)
    btn.click()
    time.sleep(20)

    browser = scrollDown(driver, 10)
    time.sleep(10)

    browser = scrollDown(driver, 10)
    time.sleep(10)

    browser = scrollDown(driver, 10)
    time.sleep(10)

    browser = scrollDown(driver, 10)

    element = driver.find_elements_by_class_name('css-ix8km1')

    subpageURL = []
    for a in element:
        subURL = a.get_attribute('href')
        subpageURL.append(subURL)

    # transform into a data frame
    dic = {'Label': ticker, 'URL': subpageURL}
    df = df.append(pd.DataFrame(dic), ignore_index = True)

# add columns
df2 = pd.DataFrame(columns=['brand', 'name', 'price', 'rank', 'skin_type', 'ingredients'])
df = pd.concat([df, df2], axis = 1)

# step 2
for i in range(len(df)+1):
    url = df.URL[i]
    driver.get(url)
    time.sleep(5)

    xpath = '/html/body/div[5]/div/div/div[1]/div/div/button'
    btn = driver.find_element_by_xpath(xpath)
    btn.click()

    # brand, name, price
    df.brand[i] = driver.find_element_by_class_name('css-avdj50').text
    df.name[i] = driver.find_element_by_class_name('css-r4ddnb ').text
    df.price[i] = driver.find_element_by_class_name('css-n8yjg7 ').text

    browser = scrollDown(driver, 1)
    time.sleep(5)
    browser = scrollDown(driver, 1)
    time.sleep(5)

    # skin_type
    detail = driver.find_element_by_class_name('css-192qj50').text
    pattern = r"✔ \w+\n"
    df.skin_type[i] = re.findall(pattern, detail)

    # ingredients
    xpath = '//*[@id="tab2"]'
    btn = driver.find_element_by_xpath(xpath)
    btn.click()

    try:
        df.ingredients[i] = driver.find_element_by_xpath('//*[@id="tabpanel2"]/div').text
    except NoSuchElementException:
        df.ingredients[i] = 'No Info'

    # rank
    try:
        rank = driver.find_element_by_class_name('css-ffj77u').text
        rank = re.match('\d.\d', rank).group()
        df['rank'][i] = str(rank)

    except NoSuchElementException:
        df['rank'][i] = 0

    print(i)    # just for verbose


df.to_csv('data/cosmetic.csv', encoding = 'utf-8-sig', index = False)